# Stacking Models 3: Weighted Average

In [1]:
# compare ensemble to each standalone models for regression
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import StackingRegressor
from matplotlib import pyplot
 
# get the dataset
# a simulated dataset
def get_dataset():
	X, y = make_regression(n_samples=1000, n_features=20, n_informative=15, noise=0.1, random_state=1)
	return X, y

In [2]:
# define dataset
X, y = get_dataset()
print(X.shape)
print(X[:2])
print(y[:2])

(1000, 20)
[[ 0.59332206 -0.56637507  1.34808718 -0.57054047 -0.72480487  1.05648449
   0.77744852  0.07361796  0.88398267  2.02843157  1.01902732  0.11227799
   0.94218853  0.26741783  0.91458143 -0.72759572  1.08842814 -0.61450942
  -0.69387293  1.69169009]
 [ 0.02335703  1.31289541 -0.10889847  1.29099337  0.88608992  1.04003845
  -1.56111989  0.26486763  0.3093178   0.07983961 -0.8311734  -0.39712568
   0.31897183  0.52795055 -0.56398646  0.93755484  2.91730876  0.57326545
   1.15321262  1.0986885 ]]
[419.49285417 524.80418832]


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [4]:
#define level0 models
model_0 = KNeighborsRegressor()
model_1 = KNeighborsRegressor(n_neighbors=3)
model_2 = KNeighborsRegressor(n_neighbors=10)

model_3 = DecisionTreeRegressor(max_depth=3)
model_4 = DecisionTreeRegressor(max_leaf_nodes=6)

model_5 = SVR()

In [5]:
models = [model_0,model_1,model_2,model_3,model_4,model_5]
y_train.shape
import numpy as np
pred_trains = np.empty((y_train.shape[0],1), int)

pred_trains.shape

(750, 1)

In [6]:
import numpy as np
from sklearn.metrics import mean_squared_error as mse

pred_trains = np.empty((y_train.shape[0],1), int)

for model in models:
    print(model)
    model.fit(X_train,y_train)
    #print(model.score(X_train,y_train))
    pred_y= model.predict(X_train).reshape(y_train.shape[0],1)
    print(mse(y_train, pred_y, squared=False))
    pred_trains = np.append(pred_trains, pred_y, axis=1)
    print(pred_trains.shape)
pred_trains = pred_trains[:,1:]

KNeighborsRegressor()
103.3874876516344
(750, 2)
KNeighborsRegressor(n_neighbors=3)
97.29476723203146
(750, 3)
KNeighborsRegressor(n_neighbors=10)
110.71301618330673
(750, 4)
DecisionTreeRegressor(max_depth=3)
165.81415340556347
(750, 5)
DecisionTreeRegressor(max_leaf_nodes=6)
173.53825514826596
(750, 6)
SVR()
205.45871176333384
(750, 7)


In [7]:
print(pred_trains[1:3])
print(y_train[1:3])

[[ 11.6504239   57.81979212  24.96050309 177.87293708 177.87293708
    2.61604134]
 [ 54.00919326 107.58159401  40.87665889 177.87293708 177.87293708
    6.09579783]]
[-41.77624922  66.61585282]


# Combine two models using weighted average

In [8]:
prediction_1 = pred_trains[:,0]
prediction_2 = pred_trains[:,3]
xx = prediction_1 - prediction_2
yy = y_train - prediction_2

yy.shape

(750,)

In [9]:
weighted_model = LinearRegression(fit_intercept=False) # no intercept
weighted_model.fit(xx.reshape(-1,1), yy.reshape(-1,1))
weighted_model.coef_[0,0]

0.9869339697963488

In [10]:
w = weighted_model.coef_[0,0]
weighted_prediction_train = w * prediction_1 + (1-w) * prediction_2
mse(weighted_prediction_train,y_train, squared=False)

103.37323922078103

In [11]:
pred_tests = np.empty((y_test.shape[0],1), int)

for model in models:
    print(model)
    pred_y= model.predict(X_test).reshape(y_test.shape[0],1)
    #print(model.score(X_test,y_test))
    print(mse(y_test, pred_y, squared=False))
    pred_tests = np.append(pred_tests, pred_y, axis=1)
pred_tests = pred_tests[:,1:]
    

print("weighted predition")

weighted_prediction_test = w * pred_tests[:,0] + (1-w) * pred_tests[:,3]

print(mse(weighted_prediction_test,y_test, squared=False))

KNeighborsRegressor()
132.39152988754225
KNeighborsRegressor(n_neighbors=3)
140.29292777086698
KNeighborsRegressor(n_neighbors=10)
127.2191779890974
DecisionTreeRegressor(max_depth=3)
168.3515561530396
DecisionTreeRegressor(max_leaf_nodes=6)
173.35417706547514
SVR()
203.02441237824215
weighted predition
131.98211706827556


# Add a third model

In [12]:
prediction_3 = pred_trains[:,2]
xx = weighted_prediction_train - prediction_3
yy = y_train - prediction_3

weighted_model_3 = LinearRegression(fit_intercept=False) # no intercept
weighted_model_3.fit(xx.reshape(-1,1), yy.reshape(-1,1))
u = weighted_model_3.coef_[0,0]
print(u)

weighted_prediction_train_3 = u * weighted_prediction_train + (1-u) * prediction_3
print(mse(weighted_prediction_train_3,y_train, squared=False))

0.7902455180047285
102.79557474917775


In [13]:
weighted_prediction_test_3 = u * weighted_prediction_test + (1-u) * pred_tests[:,2]
print(mse(weighted_prediction_test_3,y_test, squared=False))

129.40900680205786
